In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Diron\\Desktop\\EduGuideAI\\EduGuideAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Diron\\Desktop\\EduGuideAI\\EduGuideAI'

In [ ]:
# Ensure this cell does not crash the notebook when packages are missing.
# If you want to install into the currently selected kernel, run the pip line manually.
# %pip install langchain langchain_community langchain_text_splitters --quiet

# Guarded imports for document loaders and text splitter (avoids hard failures in the editor)
try:
    from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
except Exception:
    PyPDFLoader = None
    DirectoryLoader = None

try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    try:
        from langchain_text_splitters import RecursiveCharacterTextSplitter
    except Exception:
        RecursiveCharacterTextSplitter = None


Note: you may need to restart the kernel to use updated packages.


c:\Users\Diron\miniconda3\New folder\envs\EduGuideAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Extract Data From the PDF File (defensive: returns empty list if loaders missing)
def load_pdf_file(data):
    if DirectoryLoader is None or PyPDFLoader is None:
        print("PDF loader is not available in this environment.")
        return []
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    try:
        documents = loader.load()
    except Exception as e:
        print("Failed to load PDFs:", e)
        return []

    return documents

In [ ]:
# Load PDFs defensively: if no files or loader missing, extracted_data will be an empty list
try:
    extracted_data = load_pdf_file(data='Data/')
except Exception as e:
    print('Could not extract PDF data:', e)
    extracted_data = []


In [ ]:
#extracted_data

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00', 'source': 'Data\\Edu.pdf', 'total_pages': 1120, 'page': 0, 'page_label': '1'}, page_content='Faculty of Information Technology \nUniversity of Moratuwa \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nStudent Handbook Batch 22'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00', 'source': 'Data\\Edu.pdf', 'total_pages': 1120, 'page': 1, 'page_label': '2'}, page_content='Faculty of Information Technology \nUniversity of Moratuwa \n \n \n \n \n \n \n \nStudent Handbook Batch 22 \n \n \n \n \nBachelor of Science Honours in Information Technology  \n \nBachelor of Science Honours in Information Technology & Management  \n \nBachelor of Science Honours in Artificial Intelligence'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5300


In [ ]:
# Normalize text_chunks into a list of document-like objects so later code can rely on page_content
class _SimpleDoc:
    def __init__(self, text):
        self.page_content = text

# If text_chunks isn't defined or is empty, ensure it's a list
try:
    _ = text_chunks
except NameError:
    text_chunks = []

# Wrap plain strings into SimpleDoc and pass through existing objects
normalized = []
for t in text_chunks:
    if isinstance(t, str):
        normalized.append(_SimpleDoc(t))
    else:
        # try to detect common Document shape
        if hasattr(t, 'page_content') or hasattr(t, 'content'):
            normalized.append(t)
        else:
            try:
                normalized.append(_SimpleDoc(str(t)))
            except Exception:
                pass

text_chunks = normalized
print('Normalized text_chunks length ->', len(text_chunks))


[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00', 'source': 'Data\\Edu.pdf', 'total_pages': 1120, 'page': 0, 'page_label': '1'}, page_content='Faculty of Information Technology \nUniversity of Moratuwa \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nStudent Handbook Batch 22'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00', 'source': 'Data\\Edu.pdf', 'total_pages': 1120, 'page': 1, 'page_label': '2'}, page_content='Faculty of Information Technology \nUniversity of Moratuwa \n \n \n \n \n \n \n \nStudent Handbook Batch 22 \n \n \n \n \nBachelor of Science Honours in Information Technology  \n \nBachelor of Science Honours in Information Technology & Management  \n \nBachelor of Science Honours in Artificial Intelligence'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-08T05:42:08+00:00

In [10]:
# Embeddings provider import is deferred to the factory below so we can
# provide clear fallbacks and helpful error messages at runtime.
# (Do not import HuggingFaceEmbeddings at top-level; it can trigger large
# dependency imports like torch/transformers.)

In [13]:
# Factory to create embeddings. Tries local HuggingFace (sentence-transformers)
# first, and falls back to OpenAIEmbeddings if OPENAI_API_KEY is present.
def download_hugging_face_embeddings():
    """Return an embeddings object.
    Tries HuggingFace local embeddings first (needs sentence-transformers + compatible huggingface-hub).
    If that fails and OPENAI_API_KEY is set, will return OpenAIEmbeddings().
    Otherwise raises a RuntimeError with instructions."""
    import os
    try:
        # Defer heavy imports until they are actually needed
        from langchain_community.embeddings import HuggingFaceEmbeddings
        return HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    except Exception as exc:
        # Try OpenAI fallback (fast if user has a key)
        if os.environ.get('OPENAI_API_KEY'):
            try:
                # langchain versions expose OpenAI embeddings from different modules
                try:
                    from langchain.embeddings import OpenAIEmbeddings
                except Exception:
                    from langchain_openai import OpenAIEmbeddings as OpenAIEmbeddings
                return OpenAIEmbeddings()
            except Exception:
                pass
        # No fallback available: raise an informative error
        raise RuntimeError("Could not create local HuggingFace embeddings (missing or incompatible packages).") from exc


In [ ]:
# Create embeddings object defensively. The factory may raise a RuntimeError
# if local HF packages are missing; we prefer to continue and provide an
# informative message so you can choose OpenAI fallback or install locally.
try:
    embeddings = download_hugging_face_embeddings()
except RuntimeError as e:
    print('Could not create local HuggingFace embeddings:', e)
    embeddings = None
    print('If you want to use cloud embeddings, set OPENAI_API_KEY and re-run this cell.')
except Exception as e:
    print('Unexpected error while creating embeddings:', type(e).__name__, e)
    embeddings = None

# Provide a tiny fallback embeddings implementation so downstream code can run
if embeddings is None:
    class SimpleEmbeddings:
        def __init__(self, dim=384):
            self.dim = dim
        def embed_query(self, text):
            # return a deterministic, small vector (not useful semantically but keeps the flow)
            import hashlib, struct
            h = hashlib.md5(text.encode('utf8')).digest()
            vals = [b/255.0 for b in h]
            # pad or trim to dim
            vec = (vals * (self.dim // len(vals) + 1))[:self.dim]
            return vec
        def embed_documents(self, docs):
            return [self.embed_query(getattr(d, 'page_content', str(d))) for d in docs]
    embeddings = SimpleEmbeddings()
    print('Using SimpleEmbeddings fallback (non-semantic).')


In [16]:
query_result = embeddings.embed_query("What is AI?")
print("Length", len(query_result))    

Length 384


In [27]:
#query_result 

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')


In [ ]:
# Create Pinecone index if API key is present; otherwise skip with a message
try:
    from pinecone.grpc import PineconeGRPC as Pinecone
    from pinecone import ServerlessSpec
except Exception as e:
    print('pinecone package not available:', e)
    Pinecone = None

if Pinecone is None or not PINECONE_API_KEY:
    print('Skipping Pinecone index creation: missing package or PINECONE_API_KEY')
else:
    try:
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index_name = "eduguideai"
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
        print('Pinecone index ensured:', index_name)
    except Exception as e:
        print('Failed to create Pinecone index:', type(e).__name__, e)


{
    "name": "eduguideai",
    "metric": "cosine",
    "host": "eduguideai-ff8gz22.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index or a local fallback.
try:
    from langchain_pinecone import PineconeVectorStore
except Exception:
    PineconeVectorStore = None

if PineconeVectorStore is not None and embeddings is not None and PINECONE_API_KEY:
    try:
        docsearch = PineconeVectorStore.from_documents(
            documents=text_chunks,
            index_name=index_name,
            embedding=embeddings,
        )
    except Exception as e:
        print('PineconeVectorStore failed, falling back to local store:', e)
        docsearch = None
else:
    # Local in-memory vector store fallback
    class SimpleRetriever:
        def __init__(self, docs):
            self.docs = docs
        def get_relevant_documents(self, query, k=3):
            # naive retrieval: return first k docs
            return self.docs[:k]
        def invoke(self, query):
            return self.get_relevant_documents(query)
    class SimpleDocSearch:
        def __init__(self, docs):
            self._docs = docs
        def as_retriever(self, search_type='similarity', search_kwargs=None):
            return SimpleRetriever(self._docs)
    # Ensure documents are Document-like objects
    docs = text_chunks or []
    docsearch = SimpleDocSearch(docs)
    print('Using local SimpleDocSearch fallback with', len(docs), 'documents')


In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [23]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})    

In [25]:
retrieved_docs = retriever.invoke("How to apply hostel?")

In [26]:
retrieved_docs

[Document(id='e49da01c-e120-46dd-8018-7cef5ab755f5', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-12-08T05:42:08+00:00', 'page': 84.0, 'page_label': '85', 'producer': 'iLovePDF', 'source': 'Data\\Edu.pdf', 'total_pages': 1120.0}, page_content='year and the ﬁnal year students are given priority and selection will be made on criteria \napproved by the University. All resident students should pay the fees prescribed by the \nUniversity. The receipt of payment should be forwarded to the hostel sub-wardens. All \nresidential students should follow the stipulated rules and regulations. \n \n5.3. Administration of Hostels \nForth administration of hostels, two senior members of the academic staff have'),
 Document(id='ffb959cd-1471-438f-83eb-c69348804ee7', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-12-08T05:42:08+00:00', 'page': 602.0, 'page_label': '603', 'producer': 'iLovePDF', 'source': 'Data\\Edu.pdf', 'total_pages': 1120.0}, page_content='HOS

In [61]:
# Create an LLM (OpenAI if available, otherwise a lightweight fallback)
try:
    from langchain_openai import OpenAI
    llm = OpenAI(temperature=0.4, max_tokens=500)
except Exception:
    try:
        from langchain import OpenAI as OpenAIAlt  # some installs expose OpenAI here
        llm = OpenAIAlt(temperature=0.4, max_tokens=500)
    except Exception:
        class DummyLLM:
            def __init__(self, prefix='[LLM] '):
                self.prefix = prefix
            def __call__(self, prompt):
                return self.prefix + 'This is a dummy response for prompt length ' + str(len(prompt))
            def predict(self, prompt):
                return self.__call__(prompt)
        llm = DummyLLM()
        print('Using DummyLLM fallback (no OpenAI available).')


In [62]:
# ----------------------------
# 1️⃣ Prompt Templates
# ----------------------------
from typing import List, Any

try:
    from langchain.prompts.chat import (
        ChatPromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate,
    )
except Exception:
    # Minimal fallback implementations
    class SystemMessagePromptTemplate:
        def __init__(self, template): self.template = template
        @classmethod
        def from_template(cls, template): return cls(template)
        def format(self, **kwargs): return self.template.format(**kwargs)

    class HumanMessagePromptTemplate:
        def __init__(self, template): self.template = template
        @classmethod
        def from_template(cls, template): return cls(template)
        def format(self, **kwargs): return self.template.format(**kwargs)

    class ChatPromptTemplate:
        def __init__(self, messages): self.messages = messages
        @classmethod
        def from_messages(cls, messages): return cls(messages)
        def format(self, **kwargs):
            parts = [m.format(**kwargs) if hasattr(m, "format") else str(m) for m in self.messages]
            return "\n\n".join(parts)

# ----------------------------
# 2️⃣ System Prompt
# ----------------------------
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

# ----------------------------
# 3️⃣ QA Chain
# ----------------------------
def create_stuff_documents_chain(llm, prompt_template):
    def chain(docs: List[Any], input_question: str) -> str:
        texts = [getattr(d, "page_content", None) or getattr(d, "content", None) or str(d) for d in docs]
        context = "\n\n".join(texts)

        # If no relevant documents found
        if not context.strip():
            return "I don’t know the answer to that."

        prompt_text = prompt_template.format(context=context) + "\n\nQuestion: " + input_question + "\nAnswer:"
        try:
            result = llm(prompt_text)
            if isinstance(result, dict) and "text" in result:
                return result["text"]
            return str(result)
        except Exception:
            return "I don’t know the answer to that."

    return chain

# ----------------------------
# 4️⃣ RAG Chain
# ----------------------------
def create_retrieval_chain(retriever, qa_chain):
    class RAG:
        def __init__(self, retriever, qa_chain):
            self.retriever = retriever
            self.qa_chain = qa_chain

        def invoke(self, inputs):
            question = inputs.get("input") if isinstance(inputs, dict) else inputs
            docs = []

            # Try retriever methods
            try:
                if hasattr(self.retriever, "get_relevant_documents"):
                    docs = self.retriever.get_relevant_documents(question)
                elif hasattr(self.retriever, "invoke"):
                    docs = self.retriever.invoke(question)
            except Exception:
                docs = []

            # Fallback if no documents retrieved
            if not docs:
                return {"answer": "I don’t know the answer to that."}

            answer = self.qa_chain(docs, question)

            # Fallback if QA chain returns empty
            if not answer or not answer.strip():
                return {"answer": "I don’t know the answer to that."}

            return {"answer": answer}

    return RAG(retriever, qa_chain)


In [63]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [69]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

def invoke_rag_safe(rag, query):
	"""
	Robustly invoke rag.invoke accepting either a dict payload or a raw string.
	Handles implementations that expect inputs["input"] or inputs.get("input").
	Returns a dict-like response or a fallback dict with an error message.
	"""
	payload = query if isinstance(query, dict) else {"input": query}
	try:
		return rag.invoke(payload)
	except KeyError:
		# Some RAG implementations expect the raw query instead
		try:
			return rag.invoke(query)
		except Exception as e:
			return {"answer": f"Invocation failed: {type(e).__name__}: {e}"}
	except Exception as e:
		return {"answer": f"Invocation failed: {type(e).__name__}: {e}"}

for q in ["give information about MIT course", "tell me the secret of the universe"]:
	resp = invoke_rag_safe(rag_chain, q)
	# print answer safely whether resp is dict-like or plain string
	if isinstance(resp, dict):
		print(resp.get("answer", str(resp)))
	else:
		print(str(resp))


Invocation failed: KeyError: 'input'
Invocation failed: KeyError: 'input'
Invocation failed: KeyError: 'input'


In [70]:
response = rag_chain.invoke({"input": "How to apply hostel?"})
print(response["answer"])

KeyError: 'input'

In [71]:
response = rag_chain.invoke({"input": "How to drink?"})
print(response["answer"])

KeyError: 'input'

In [ ]:
response = rag_chain.invoke({"input": "0"})
print(response["answer"])

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise. 

75

72

65

Question: 0
Answer:


In [7]:
response = rag_chain.invoke({"input": "give information about mit course"})
print(response["answer"])

NameError: name 'rag_chain' is not defined

In [11]:
response = rag_chain.invoke({"input": "how many courses in university of kelaniya"})
print(response["answer"])

NameError: name 'rag_chain' is not defined

In [72]:
response = rag_chain.invoke({"input": "give information about mit course"})
print(response["answer"])

KeyError: 'input'

In [32]:
response = rag_chain.invoke({"input": "give information about mit course"})
print(response["answer"])

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise. 

Note:  1. Some of the optional course units will be offered depending on the staff availability. Students are 
requested to consult the Head of the Department prior to their registrations for the Year of Study 3 and 
Year of Study 4 course units. 
 
2. A student who follows BSc Hons in MIT/IT Degree programmes may take any course units offered by 
the Department of Industrial Management and Axillary course units offered by any other Faculties/

Handbook-2023/2024, Faculty of Science, University of Kelaniya 
29 
To be eligible for the Higher Diploma in MIT/ Higher Diploma in IT, a student 
must 
(i) accumulate grades of D or better in course units including all 
compulsory course units, aggregating to at least 72 credits from year 
of study 1 and year of s

In [42]:
def create_stuff_documents_chain(llm, prompt_template):
    def chain(docs, input_question):

        # Extract context text
        texts = []
        for d in docs:
            text = getattr(d, "page_content", None) or getattr(d, "content", None)
            if text:
                texts.append(text)

        # If no documents → question irrelevant
        if len(texts) == 0:
            return "I don't know the answer based on the available documents."

        context = "\n\n".join(texts)

        # Build prompt
        final_prompt = prompt_template.format(
            context=context,
            input=input_question
        )

        # Call LLM safely
        try:
            response = llm(final_prompt)
            return response.strip()
        except:
            if hasattr(llm, "predict"):
                return llm.predict(final_prompt).strip()

        return "I could not generate a response."
    return chain


In [43]:
def create_retrieval_chain(retriever, qa_chain):
    class RAG:
        def __init__(self, retriever, qa_chain):
            self.retriever = retriever
            self.qa_chain = qa_chain

        def invoke(self, inputs):
            question = inputs["input"]

            # Retrieve relevant docs
            docs = self.retriever.get_relevant_documents(question)

            # Get answer
            answer = self.qa_chain(docs, question)

            return {"answer": answer}
    return RAG(retriever, qa_chain)


In [48]:
response = rag_chain.invoke({"input": "give information about mit course"})
print(response["answer"])

AttributeError: 'VectorStoreRetriever' object has no attribute 'get_relevant_documents'